# TinyLoRA 2D SFT Loss Surface (Budget=2)

Scans a 20x20 grid over the two TinyLoRA trainable scalars and visualizes the SFT loss landscape.


In [ ]:
from pathlib import Path
PROJECT_ROOT = Path('..').resolve()
%cd {PROJECT_ROOT}
print('PROJECT_ROOT =', PROJECT_ROOT)


In [ ]:
!python -m pip install -q -r requirements.txt


In [ ]:
!python scripts/analysis/scan_loss_surface.py \
  --root-dir ./outputs/loss_surface_b2_20x20 \
  --model-name Qwen/Qwen2.5-7B-Instruct \
  --budget 2 \
  --grid-size 20 \
  --vmin -0.05 \
  --vmax 0.05 \
  --train-limit 128 \
  --batch-size 4 \
  --max-length 512 \
  --dtype bfloat16 \
  --seed 1 \
  --dataset-name gsm8k \
  --dataset-config main \
  --train-split train \
  --question-field question \
  --answer-field answer \
  --resume


In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

run_dir = Path('outputs/loss_surface_b2_20x20')
csv_path = run_dir / 'loss_surface.csv'
if not csv_path.exists():
    candidates = sorted(run_dir.glob('loss_surface_*.csv'), key=lambda p: p.stat().st_mtime, reverse=True)
    assert candidates, f'No loss-surface CSV in {run_dir}'
    csv_path = candidates[0]

df = pd.read_csv(csv_path)
assert not df.empty
pivot = df.pivot(index='i', columns='j', values='loss').sort_index().sort_index(axis=1)
Z = pivot.values

best = df.loc[df['loss'].idxmin()]
worst = df.loc[df['loss'].idxmax()]
print(f'Rows: {len(df)}')
print(f"Best loss={best['loss']:.6f} at (p1={best['p1']:.6f}, p2={best['p2']:.6f})")
print(f'Dynamic range={float(worst['loss'] - best['loss']):.6f}')

p1_vals = np.sort(df['p1'].unique())
p2_vals = np.sort(df['p2'].unique())
X, Y = np.meshgrid(p2_vals, p1_vals)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
im = axes[0].imshow(Z, origin='lower', aspect='auto', cmap='viridis')
axes[0].set_title('Loss Heatmap')
axes[0].set_xlabel('j (p2 index)')
axes[0].set_ylabel('i (p1 index)')
axes[0].scatter([best['j']], [best['i']], c='red', s=50, marker='x')
fig.colorbar(im, ax=axes[0], fraction=0.046, pad=0.04)

cs = axes[1].contourf(X, Y, Z, levels=20, cmap='viridis')
axes[1].contour(X, Y, Z, levels=12, colors='white', linewidths=0.5, alpha=0.6)
axes[1].scatter([best['p2']], [best['p1']], c='red', s=50, marker='x')
axes[1].set_title('Loss Contours (p1/p2)')
axes[1].set_xlabel('p2')
axes[1].set_ylabel('p1')
fig.colorbar(cs, ax=axes[1], fraction=0.046, pad=0.04)
fig.tight_layout()
png_path = run_dir / 'loss_surface.png'
fig.savefig(png_path, dpi=160)
plt.show()
print('Saved:', png_path)
